In [3]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib notebook
from datetime import datetime
from sklearn.linear_model import LinearRegression

import h5py
import json
import pickle

C:\Users\sukan\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [63]:
import math

In [5]:
# Open file containing list of all restaurant business IDs

with open('C:/Users/sukan/Documents/TUM/Courses/Winter 2018/Data Mining Praktikum/Yelp/restaurants_considered_business_ids.pkl', 'rb') as f:
    restaurants_considered_business_ids = pickle.load(f)
f.close()

In [6]:
# Open Business File

with open('C:/Users/sukan/Documents/TUM/Courses/Winter 2018/Data Mining Praktikum/Yelp/dataset/yelp_dataset/yelp_academic_dataset_business.json', encoding="utf8") as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 
    
business_data = pd.DataFrame(data)
json_file.close()

In [12]:
# Open Reviews File

with open('C:/Users/sukan/Documents/TUM/Courses/Winter 2018/Data Mining Praktikum/Yelp/dataset/yelp_dataset/yelp_academic_dataset_review.json', encoding="utf8") as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 
    
review_data = pd.DataFrame(data)
json_file.close()

In [13]:
# Extract only reviews of restaurants

restuarant_review_data = review_data.loc[review_data['business_id'].isin(restaurant_ids)]

In [7]:
# Extract only considered restaurants - All open restaurants + restaurants closed within the last 3 months

considered_restuarant_business_data = business_data.loc[business_data['business_id'].isin(restaurants_considered_business_ids)]

In [61]:
# Extract only considered restaurants - All open restaurants + restaurants closed within the last 3 months

restaurant_data = business_data.loc[business_data['business_id'].isin(restaurant_ids)]

In [8]:
len(considered_restuarant_business_data)

41795

In [9]:
considered_restuarant_business_data.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,None,1,43.712946,-79.632763,Thai One On,Ridgewood,L4T 1A8,7,2.0,ON
8,1440 N. Dysart Ave,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.448106,-112.341302,Filiberto's Mexican Food,,85323,40,2.5,AZ
12,2801 N 15th Ave,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, Dive Bars, Burgers, Nightli...",Phoenix,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,33.479807,-112.091188,Original Hamburger Works,,85007,277,4.0,AZ
13,4216 Saint-Laurent Boul,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",bBUMib8l6Me1ZB1_Qkezkg,"Restaurants, Pakistani, Indian, Middle Eastern",Montréal,None,1,45.518036,-73.582174,Mysore Indian Cuisine,Plateau-Mont-Royal,H2W 1Z3,19,3.5,QC


In [11]:
considered_restuarant_f = considered_restuarant_business_data.copy()

In [15]:
restuarant_review_data['date'] = pd.to_datetime(restuarant_review_data['date'])

C:\Users\sukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
# restuarant_review_data.sort_values(by=['date'], ascending=False)

In [18]:
from numpy import datetime64

date_limit = datetime64('2018-04-01')

restuarant_review_data_considered = restuarant_review_data[restuarant_review_data.date < date_limit]

In [19]:
restuarant_review_data_considered.sort_values(by=['date'], ascending=False)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
3726226,mXsumyOgPP-cRQuFQmRivg,0,2018-03-31,0,j2orXpuZXv7KP2iT2qPqHw,4,This restaurant is in a very charming setting....,0,N7Kf9V4tbyvw93C9CZ4Kxg
2090926,saijdv-vXJrvsCfvr7SZOw,0,2018-03-31,0,a4nDGZP9Qz7gZr43uGkwUw,3,We've tried few things from their menu and fou...,2,I4eBZixB6oMmyCCR0EChgA
1702288,nD8AQZldZw0aBs3h7HxNMA,0,2018-03-31,0,Oo0Q42lLVyDNOO84EInOPw,5,Amazing tasting food and the prices are reason...,0,ArbcVw_-RJZXUMm0NL1XbA
5312008,X88w0XrUSIKue9Vuwtqs_g,0,2018-03-31,0,M9ZTGeWW5YsGqE0DJuuz6g,5,"Caught the Saturday happy hour (4 to 6, same a...",0,gFI1lEppec7QJAmdFdPQjg
4641600,Qn_mBXvKnn9VeOKym0Ussw,0,2018-03-31,0,eGpqe_ZwaT27cYKdyAJ3_g,5,We love love this place! It's our go to especi...,0,kOcmoDREmcJ6Knp9VUb2Fg
4641593,6tSvz_21BMo3a4GaItwa0g,0,2018-03-31,0,pyAGXYfoQ52kC8BrxyAg1w,3,I really wanted to like this place especially ...,0,kOcmoDREmcJ6Knp9VUb2Fg
1010372,CvonRhKDJaH155xhtpz_iw,0,2018-03-31,0,RIymsD1SHXZAO8VUVqFoiw,5,"We stopped in on a Friday night, so it was bus...",0,QTQL0mbrOKWGp0GTYF-4ww
1453577,0_aeYE2-VbsZts_UpILgDw,0,2018-03-31,0,sJNmYrNSMfOKywYqbvZC_w,1,Another good one gone :( I'm a vegetarian and ...,0,kTny8RFBPj9du2aIRb8V0A
3914289,5WOE18ldtQ1U0MK5fN2HGQ,0,2018-03-31,0,MlsFknwaJ4VtTMWt1gG1wg,1,This place was terrible. They messed up my wif...,1,KIM_AVOpSx3McxFa8fv9MA
5145441,3H48sIcoOooMFsTME0Jm_Q,0,2018-03-31,0,7AJsINMqo5LI18EuaN0L_Q,5,This place is amazing. The decorations are so ...,0,-zJSoF4et3Rm3OZL8bhM5w


In [21]:
today = date_limit

In [24]:
restuarant_review_data_considered['date'] = pd.to_datetime(restuarant_review_data_considered['date'])

C:\Users\sukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
restuarant_review_data_considered.date.dtype

dtype('<M8[D]')

In [29]:
a = np.datetime64(max(restuarant_review_data_considered.date))

In [42]:
b = np.datetime64(min(restuarant_review_data_considered.date))

In [43]:
b

numpy.datetime64('2004-10-12T00:00:00.000000')

In [46]:
(today)

numpy.datetime64('2018-04-01')

In [45]:
(today - b)/(24*60*60*1000000)

numpy.timedelta64(4919,'us')

In [58]:
oldest_date = datetime64('2016-04-01')

In [65]:
def distance(ilat,jlat,ilong,jlong):
    R = 6371.e3 # earth radius in m
    phi1 = math.radians(ilat)
    phi2 = math.radians(jlat)
    deltaphi = math.radians(jlat-ilat)
    deltalambda = math.radians(jlong-ilong)
    a = math.sin(deltaphi/2)**2 + math.cos(phi1)*math.cos(phi2) * math.sin(deltalambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c / 1609.34 # in miles
    return d

In [70]:
reviews_to_be_considered = {2016: [4, 5, 6, 7, 8, 9, 10, 11, 12],
2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
2018: [1, 2, 3]}

In [71]:
reviews_to_be_considered

{2016: [4, 5, 6, 7, 8, 9, 10, 11, 12],
 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2018: [1, 2, 3]}

In [78]:
month_numbers = [0, 1, 2, 3, 4, 5,6 ,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
print (month_numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [85]:
considered_restuarant_business_data = considered_restuarant_business_data.reset_index()

In [106]:
final_restaurant_data_for_model = pd.DataFrame(columns=['age', 'restaurant_density', 'stars_coeff', 'review_cnt_permonth_coeff', 'no_of_reviews_in_last_2_yrs', 'is_open'])

In [107]:
final_restaurant_data_for_model

,age,restaurant_density,stars_coeff,review_cnt_permonth_coeff,no_of_reviews_in_last_2_yrs,is_open


In [104]:
from datetime import datetime
from sklearn.linear_model import LinearRegression

stars_regr = LinearRegression()
review_cnt_regr = LinearRegression()
# considered_restuarant_business_data['reviews_per_week'] = np.nan
# considered_restuarant_business_data['oldest_review'] = np.nan # expressed in days from today
# considered_restuarant_business_data['std_of_stars'] = np.nan
# considered_restuarant_business_data['median_of_stars'] = np.nan
# considered_restuarant_business_data['stars_linear_coef'] = np.nan

cnt = 0
# 
for i, restaurant in considered_restuarant_business_data.iterrows():
    if (i==5):
        break
    restaurant_id = restaurant['business_id']
    reviews = restuarant_review_data_considered[restuarant_review_data_considered['business_id'] == restaurant_id]
    oldest_review = np.datetime64(min(reviews.date))
    age = int(str((today - oldest_review)/(24*60*60*1000000)).strip(' microseconds'))
    
    relevant_reviews = reviews.loc[reviews.date >= oldest_date]
    ilat = restaurant.latitude
    ilong = restaurant.longitude
    restaurant_density = 0
    for j, jrestaurant in restaurant_data.iterrows():
        jlong = jrestaurant['longitude']
        jlat = jrestaurant['latitude']
        dist = distance(ilat,jlat,ilong,jlong)
        # if less than 1 mile (include self in calculation of statistics)
        if dist <= 1:
            restaurant_density += 1
    if (len(relevant_reviews)<10):
        continue
        
    cnt = cnt + 1
    print ('---------------------' + str(i) + '---------------------')
    print ('---------------------' + restaurant_id + '---------------------')
    print('Restaurant Density: ' + str(restaurant_density))
    if (restaurant.is_open==1):
        print ('----Restaurant is open')
    else:
        print ('----Restaurant is closed')
#     print(len(reviews))
#     print(len(relevant_reviews))
    
    dates = [d.toordinal() for d in relevant_reviews.date] # convert dates to integer
    dates = np.asarray(dates)
#     print(dates)
    stars_regr.fit(dates.reshape(-1,1),relevant_reviews.stars.values.reshape(-1,1)) 
    
    stars_coeff = stars_regr.coef_[0][0]
    print('Stars Coeff' + str(stars_coeff))
    review_cnts = []
    
    for year in reviews_to_be_considered.keys():
        months = reviews_to_be_considered[year]
        for month in months:
            review_cnts.append(len(relevant_reviews.loc[(relevant_reviews['date'].dt.month==month) & (relevant_reviews['date'].dt.year==year)]))
            
    print (review_cnts)
    
    review_cnt_regr.fit(np.array(month_numbers).reshape(-1,1),np.array(review_cnts).reshape(-1,1))
    
    review_cnt_coeff = review_cnt_regr.coef_[0][0]
    print ('Reviews Coeff' + str(review_cnt_coeff))
    print ('No of reviews in last 2 years: ' + str(no_of_reviews_in_last_2_yrs))
    print('---------------------')
    
    no_of_reviews_in_last_2_yrs = len(relevant_reviews)
    final_restaurant_data_for_model = final_restaurant_data_for_model.append({'age': age, 
                                            'restaurant_density': restaurant_density, 
                                            'stars_coeff': stars_coeff, 
                                            'review_cnt_permonth_coeff': review_cnt_coeff, 
                                            'no_of_reviews_in_last_2_yrs': no_of_reviews_in_last_2_yrs,
                                            'is_open': restaurant.is_open}, ignore_index=True)
    
#     delta_time = (max(df_reviews.date) - min(df_reviews.date)).days/7.
#     print(delta_time)
#     nreviews = len(df_reviews)
#     # proxy for oldness: weeks of oldest review since 2014 (all reviews are older than this)
#     weeks_since_older_review = (datetime.strptime('Jan 1 2014','%b %d %Y')- min(df_reviews.date)).days*7
#     nreactions = 0
#     #for j, review in df_reviews.iterrows():
#      #   nreactions += review.funny + review.useful + review.cool
#     dates = [d.toordinal() for d in df_reviews.date] # convert dates to integer
#     dates = np.asarray(dates)
#     # fit linear regression model to find coefficient (do stars increase or decrease?)
    
    
#     #considered_restuarant_business_data.loc[i,'reviews_per_week'] = nreviews / delta_time
    
#     # The following way of calculating frequency is better. Since restaurants with a small number of reviews
#     # that came at the same period would seem like they get reviews very frequently
#     considered_restuarant_business_data.loc[i,'reviews_per_week'] = nreviews / weeks_since_older_review
#     considered_restuarant_business_data.loc[i,'oldest_review'] = weeks_since_older_review
#     considered_restuarant_business_data.loc[i,'std_of_stars'] = np.std(reviews['stars'])
#     considered_restuarant_business_data.loc[i,'median_of_stars'] = np.median(reviews['stars'])
#     # same for this, use weeks_since_older review instead of delta_time
#     considered_restuarant_business_data.loc[i,'reactions_per_week'] = 14640162/ weeks_since_older_review
#     considered_restuarant_business_data.loc[i,'stars_linear_coef'] = regr.coef_[0][0]
    
#     if i % 100 == 0:
#         print(i)

---------------------0---------------------
---------------------Apn5Q_b6Nz61Tq4XzPdf9A---------------------
Restaurant Density: 38
----Restaurant is open
Stars Coeff0.0016452042347464238
[0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 1, 1, 2]
Reviews Coeff0.026521739130434777
No of reviews in last 2 years: 66
---------------------
---------------------2---------------------
---------------------8-NRKkPY1UiFXW20WXKiXg---------------------
Restaurant Density: 77
----Restaurant is open
Stars Coeff-0.0008174327170179505
[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1]
Reviews Coeff0.02956521739130434
No of reviews in last 2 years: 13
---------------------
---------------------3---------------------
---------------------_c3ixq9jYKxhLUB0czi0ug---------------------
Restaurant Density: 68
----Restaurant is open
Stars Coeff-0.0001911697529947934
[0, 1, 5, 2, 1, 5, 2, 0, 3, 4, 3, 3, 5, 0, 2, 3, 3, 1, 2, 3, 9, 3, 3, 3]
Reviews Coeff0.07913043478260866
No 

In [105]:
final_restaurant_data_for_model

,age,restaurant_density,stars_coeff,review_cnt_permonth_coeff,no_of_reviews_in_last_2_yrs
0,1807.0,38.0,0.001645,0.026522,13.0
1,3504.0,77.0,-0.000817,0.029565,10.0
2,4163.0,68.0,-0.000191,0.079130,66.0


In [77]:
range(0,24)

range(0, 24)

In [88]:
review_data[review_data.business_id=='6OuOZAok8ikONMS_T3EzXg']

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
435139,6OuOZAok8ikONMS_T3EzXg,0,2012-12-21,0,1K3I3fbsaSun9Al4ZLtAuw,5,love this place! its my new go to Thai restaur...,1,2dPAIXLPnRAdmjv5c-n1vw
530717,6OuOZAok8ikONMS_T3EzXg,0,2013-10-31,0,4XeOuvBOkOkti2R5wmnyyg,1,Part of my order was missing. There were hard ...,0,HkKKAw2PAWVxuuooFkNFVQ
1922696,6OuOZAok8ikONMS_T3EzXg,0,2014-03-01,0,4AYMyDvSbjjkIgnZqnfMYQ,1,Really slow delivery and almost no chicken in ...,1,0WRUJ_yi5mqM9Qz8nOYxeQ
2157594,6OuOZAok8ikONMS_T3EzXg,0,2013-09-14,0,BrZ_MMwFsmt5DyLBoIHnQw,1,Visited this site 3 times with Groupon. \nSi...,0,QnR9A0VHuT4rtSdgaLmIBQ
5070350,6OuOZAok8ikONMS_T3EzXg,0,2013-12-22,0,wZJK7LQ3KKPlg4wA9WXM3w,3,"This place is sub-par, but my bf swears by the...",0,LiNx18WUre9WFCEQlUhtKA
5646078,6OuOZAok8ikONMS_T3EzXg,0,2013-05-20,0,QTs4UfT3sLMkT9OVNds2ag,1,Last visit. Ever.\nAnother groupon and one mor...,0,6iZ4KZDyIimYFUN3rbRZEg
5751232,6OuOZAok8ikONMS_T3EzXg,0,2012-08-17,0,QzFzdW5otW72rkN7I1zMng,2,I had the Pad Thai with chicken and shrimp it ...,0,5JfchbYrvSLUL5_HTohjOQ


In [89]:
business_data[business_data.business_id=='6OuOZAok8ikONMS_T3EzXg']

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,None,1,43.712946,-79.632763,Thai One On,Ridgewood,L4T 1A8,7,2.0,ON
